In [3]:
import os
import json
import pandas as pd
from flask import Flask, render_template, request, redirect, url_for, flash
from werkzeug.utils import secure_filename
from PyPDF2 import PdfReader
from langchain.llms import Ollama

# Initialize Flask app
app = Flask(__name__)
app.secret_key = 'your_secret_key'  # Replace with a secure secret key

# Configure upload folder and allowed extensions
UPLOAD_FOLDER = 'uploads/'
ALLOWED_EXTENSIONS = {'pdf'}

app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER

# Initialize Ollama Llama model
llm = Ollama(model="hf.co/bartowski/Llama-3.2-1B-Instruct-GGUF:latest")

# Ensure upload folder exists
os.makedirs(UPLOAD_FOLDER, exist_ok=True)

# Function to check allowed file types
def allowed_file(filename):
    return '.' in filename and \
           filename.rsplit('.', 1)[1].lower() in ALLOWED_EXTENSIONS

# Function to extract text from PDF
def extract_text_from_pdf(pdf_path):
    reader = PdfReader(pdf_path)
    text = ""
    for page in reader.pages:
        text += page.extract_text() + "\n"
    return text

# Function to analyze resume
def analyze_resume(resume_text, job_description):
    prompt = f"""
Analyze the following resume and compare it to the given job description. 
Provide feedback on improvements to make the resume more relevant for the job
make short in bullet points.

Resume:
{resume_text}

Job Description:
{job_description}

Feedback:
"""
    response = llm(prompt)
    return response

# Home route
@app.route('/')
def index():
    return render_template('index.html')

# Route to handle form submission
@app.route('/analyze', methods=['POST'])
def analyze():
    if 'resume' not in request.files:
        flash('No resume part')
        return redirect(request.url)
    
    file = request.files['resume']
    if file.filename == '':
        flash('No selected file')
        return redirect(request.url)
    
    if file and allowed_file(file.filename):
        filename = secure_filename(file.filename)
        filepath = os.path.join(app.config['UPLOAD_FOLDER'], filename)
        file.save(filepath)
        
        # Extract text from resume
        resume_text = extract_text_from_pdf(filepath)
        
        # Get job description from form
        job_description = request.form.get('job_description', '')
        if not job_description:
            flash('Job description is required')
            return redirect(url_for('index'))
        
        # Analyze resume
        feedback = analyze_resume(resume_text, job_description)
        
        # Clean up uploaded file
        os.remove(filepath)
        
        return render_template('feedback.html', feedback=feedback)
    else:
        flash('Allowed file types are PDF')
        return redirect(request.url)

# Run the app
if __name__ == '__main__':
    app.run()


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [07/Dec/2024 21:13:12] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [07/Dec/2024 21:14:27] "POST /analyze HTTP/1.1" 200 -
[2024-12-07 21:14:29,984] ERROR in app: Exception on /analyze [POST]
Traceback (most recent call last):
  File "C:\Users\Asus\anaconda3\lib\site-packages\flask\app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\Asus\anaconda3\lib\site-packages\flask\app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\Asus\anaconda3\lib\site-packages\flask\app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "C:\Users\Asus\anaconda3\lib\site-packages\flask\_compat.py", line 39, in reraise
    raise value
  File "C:\Users\Asus\anaconda3\lib\site-packages\flask\app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\Users\Asus\anaconda3\lib\site-packages\flask\app

In [2]:
!pip install PYPDF2

  Using cached pypdf2-3.0.1-py3-none-any.whl.metadata (6.8 kB)
Using cached pypdf2-3.0.1-py3-none-any.whl (232 kB)


In [5]:

import os
import json
import pandas as pd
from flask import Flask, render_template, request, redirect, url_for, flash
from werkzeug.utils import secure_filename
from PyPDF2 import PdfReader
from langchain.llms import Ollama

# Initialize Flask app
app = Flask(__name__)
app.secret_key = 'your_secret_key'  # Replace with a secure secret key

# Configure upload folder and allowed extensions
UPLOAD_FOLDER = 'uploads/'
ALLOWED_EXTENSIONS = {'pdf'}
app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER

# Initialize Ollama Llama model
llm = Ollama(model="hf.co/bartowski/Llama-3.2-1B-Instruct-GGUF:latest")

# Ensure upload folder exists
os.makedirs(UPLOAD_FOLDER, exist_ok=True)

# Check allowed file types
def allowed_file(filename):
    return '.' in filename and filename.rsplit('.', 1)[1].lower() in ALLOWED_EXTENSIONS

# Extract text from PDF
def extract_text_from_pdf(pdf_path):
    reader = PdfReader(pdf_path)
    text = ""
    for page in reader.pages:
        page_text = page.extract_text()
        if page_text:
            text += page_text + "\n"
    return text

# Analyze the resume
def analyze_resume(resume_text, job_description):
    prompt = f"""
    Analyze the following resume and compare it to the given job description. 
    Provide feedback on improvements to make the resume more relevant for the job.

    Resume:
    {resume_text}

    Job Description:
    {job_description}

    Feedback:
    """
    response = llm(prompt)
    return response

# Home route
@app.route('/')
def index():
    return render_template('index.html')

# Handle form submission
@app.route('/analyze', methods=['POST'])
def analyze():
    if 'resume' not in request.files:
        flash('No resume part')
        return redirect(request.url)

    file = request.files['resume']
    if file.filename == '':
        flash('No selected file')
        return redirect(request.url)

    if file and allowed_file(file.filename):
        filename = secure_filename(file.filename)
        filepath = os.path.join(app.config['UPLOAD_FOLDER'], filename)
        file.save(filepath)

        # Extract text from resume
        resume_text = extract_text_from_pdf(filepath)

        # Get job description
        job_description = request.form.get('job_description', '')
        if not job_description:
            flash('Job description is required')
            os.remove(filepath)
            return redirect(url_for('index'))

        # Analyze resume
        try:
            feedback = analyze_resume(resume_text, job_description)
        except Exception as e:
            flash(f'Error during analysis: {e}')
            os.remove(filepath)
            return redirect(url_for('index'))

        # Clean up uploaded file
        os.remove(filepath)
        return render_template('feedback.html', feedback=feedback)

    else:
        flash('Allowed file types are PDF')
        return redirect(request.url)

# Run the app
if __name__ == '__main__':
    app.run()


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [07/Dec/2024 21:16:56] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [07/Dec/2024 21:17:55] "POST /analyze HTTP/1.1" 200 -
127.0.0.1 - - [07/Dec/2024 21:18:17] "GET / HTTP/1.1" 200 -
